In [26]:
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
from rasterio.mask import mask

thailand = gpd.read_file('tha_adm_shapefile/tha_admbnda_adm1_rtsd_20190221.shp')
thailand

,Shape_Leng,Shape_Area,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM1_REF,ADM1ALT1EN,ADM1ALT2EN,ADM1ALT1TH,ADM1ALT2TH,ADM0_EN,ADM0_TH,ADM0_PCODE,date,validOn,validTo,geometry
0,3.927244,0.275313,Amnat Charoen,อำนาจเจริญ,TH37,None,None,None,None,None,Thailand,ประเทศไทย,TH,2019-02-18,2019-02-21,None,"POLYGON ((104.95982 16.28359, 104.95986 16.283..."
1,1.739908,0.079210,Ang Thong,อ่างทอง,TH15,None,None,None,None,None,Thailand,ประเทศไทย,TH,2019-02-18,2019-02-21,None,"POLYGON ((100.33319 14.79853, 100.33341 14.798..."
2,2.417227,0.131339,Bangkok,กรุงเทพมหานคร,TH10,None,None,None,None,None,Thailand,ประเทศไทย,TH,2019-02-18,2019-02-21,None,"POLYGON ((100.61389 13.95462, 100.61428 13.954..."
3,4.414998,0.340784,Bueng Kan,บึงกาฬ,TH38,None,None,None,None,None,Thailand,ประเทศไทย,TH,2019-02-18,2019-02-21,None,"POLYGON ((103.40497 18.44898, 103.40619 18.448..."
4,8.701860,0.844537,Buri Ram,บุรีรัมย์,TH31,None,None,None,None,None,Thailand,ประเทศไทย,TH,2019-02-18,2019-02-21,None,"POLYGON ((102.93029 15.79514, 102.93029 15.795..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,10.021746,0.940132,Udon Thani,อุดรธานี,TH41,None,None,None,None,None,Thailand,ประเทศไทย,TH,2019-02-18,2019-02-21,None,"POLYGON ((102.05806 18.08620, 102.05835 18.085..."
73,5.086718,0.558744,Uthai Thani,อุทัยธานี,TH61,None,None,None,None,None,Thailand,ประเทศไทย,TH,2019-02-18,2019-02-21,None,"POLYGON ((99.13905 15.79655, 99.13918 15.79652..."
74,5.801730,0.674675,Uttaradit,อุตรดิตถ์,TH53,None,None,None,None,None,Thailand,ประเทศไทย,TH,2019-02-18,2019-02-21,None,"POLYGON ((101.09241 18.38167, 101.09277 18.381..."
75,4.839230,0.365448,Yala,ยะลา,TH95,None,None,None,None,None,Thailand,ประเทศไทย,TH,2019-02-18,2019-02-21,None,"POLYGON ((101.29272 6.68112, 101.29369 6.67953..."


In [25]:
shape = thailand[thailand['ADM1_TH'] == 'ขอนแก่น']

,Shape_Leng,Shape_Area,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM1_REF,ADM1ALT1EN,ADM1ALT2EN,ADM1ALT1TH,ADM1ALT2TH,ADM0_EN,ADM0_TH,ADM0_PCODE,date,validOn,validTo,geometry
16,8.994164,0.897807,Khon Kaen,ขอนแก่น,TH40,None,None,None,None,None,Thailand,ประเทศไทย,TH,2019-02-18,2019-02-21,None,"POLYGON ((102.70722 17.08713, 102.70796 17.087..."


In [17]:
import fiona
import rasterio
from rasterio.mask import mask
from rasterio.plot import show

with fiona.open('THA_adm/THA_adm1.shp') as src:
    for feature in src:
        shapes = [feature['geometry']]

with rasterio.open('output/KhonKaen_B04.tiff') as src:
    out, out_transform = mask(src,shapes,crop=True)
    show(out)

display(shapes)

ValueError: Input shapes do not overlap raster.

In [ ]:
with rasterio.open('input/KhonKaen_Right_B04.tiff') as src:
    print(src.crs)

In [ ]:
try:
  clipped_data, clipped_transform = mask(dataset=src, shapes=khonkaen.geometry, crop=True, invert=False)
except ValueError:
  print("Shapefile and raster might not overlap. Check extents or clip the raster.")
else:
  # Clipped data and transform are available for further use
  print("Raster successfully masked using Khon Kaen province.")

In [ ]:
out_meta.update({
    "driver":'Gtiff',
    "height":out_image.height,
    "width":out_image.width,
    "transform":out_transform
}
)


In [ ]:
from rasterio.merge import merge

right = rasterio.open('KhonKaen_Right_B04.tiff')
left = rasterio.open('KhonKaen_Left_B04.tiff')

dest, output_transform=merge([right,left], precision=7)

out_meta = right.meta

out_meta.update({"driver": "GTiff",
                 "height": dest.shape[1],
                 "width": dest.shape[2],
                 "transform": output_transform})

with rasterio.open("mergedRasters.tif", "w", **out_meta) as dest1:
        dest1.write(dest)

right.close()
left.close()

In [ ]:
from rasterio.plot import show
import rasterio

with rasterio.open('mergedRasters.tif') as src:
    show(src)

In [ ]:
import fiona
from rasterio.mask import mask
import rasterio

with fiona.open('THA_adm/THA_adm1.shp') as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

with rasterio.open('KhonKaen_B04.tiff') as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open("RGB.byte.masked.tif", "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
import fiona
with fiona.open('THA_adm/THA_adm1.shp') as src:
    b4 = src

print(b4.crs)

In [ ]:
from datetime import date, timedelta # To define date range of data
import requests # To define http request to be make 
import pandas as pd # Convert data received from copernicus API in easier format
import geopandas as gpd # Convert Pandas dataframe in Geo pandas will allow us to use metadata and geoemtry.
from shapely.geometry import shape # To convert raw Geometry data

In [ ]:
# copernicus User email
copernicus_user = "thabnatatom@gmail.com"
# copernicus User Password
copernicus_password = "mocdug-qazgez-dygfI1" 
# WKT Representation of BBOX of AOI
ft = "POLYGON((0 0, 0 1, 1 1, 1 0, 0 0))" 
# Sentinel satellite that you are interested in 
data_collection = "SENTINEL-2"

# Dates of search 

today =  date.today()
today_string = today.strftime("%Y-%m-%d")
yesterday = today - timedelta(days=30)
yesterday_string = yesterday.strftime("%Y-%m-%d")

print(yesterday_string)

In [ ]:
def get_keycloak(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Keycloak token creation failed. Reponse from the server was: {r.json()}"
        )
    return r.json()["access_token"]

get_keycloak(copernicus_user,copernicus_password)

In [ ]:
json_ = requests.get(
    f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' and OData.CSC.Intersects(area=geography'SRID=4326;{ft}') and ContentDate/Start gt {yesterday_string}T00:00:00.000Z and ContentDate/Start lt {today_string}T00:00:00.000Z&$count=True&$top=1000"
).json()  
p = pd.DataFrame.from_dict(json_["value"]) # Fetch available dataset
if p.shape[0] > 0 : # If we get data back
    p["geometry"] = p["GeoFootprint"].apply(shape)
    # Convert pandas dataframe to Geopandas dataframe by setting up geometry
    productDF = gpd.GeoDataFrame(p).set_geometry("geometry") 
    # Remove L1C dataset if not needed
    productDF = productDF[~productDF["Name"].str.contains("L1C")] 
    print(f" total L2A tiles found {len(productDF)}")
    productDF["identifier"] = productDF["Name"].str.split(".").str[0]
    allfeat = len(productDF) 
    
    if allfeat == 0: # If L2A tiles are not available in current query
        print(f"No tiles found for {today_string}")
    else: # If L2A tiles are available in current query
        # download all tiles from server
        for index,feat in enumerate(productDF.iterfeatures()):
            try:
                # Create requests session 
                session = requests.Session()
                # Get access token based on username and password
                keycloak_token = get_keycloak(copernicus_user,copernicus_password)
                
                session.headers.update({"Authorization": f"Bearer {keycloak_token}"})
                url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products({feat['properties']['Id']})/$value"
                response = session.get(url, allow_redirects=False)
                while response.status_code in (301, 302, 303, 307):
                    url = response.headers["Location"]
                    response = session.get(url, allow_redirects=False)
                print(feat["properties"]["Id"])
                file = session.get(url, verify=False, allow_redirects=True)

                with open(
                    f"location/to/save/{feat['properties']['identifier']}.zip", #location to save zip from copernicus 
                    "wb",
                ) as p:
                    print(feat["properties"]["Name"])
                    p.write(file.content)
            except:
                print("problem with server")
else : # If no tiles found for given date range and AOI
    print('no data found')

In [ ]:
import openeo

# First, we connect to the back-end and authenticate. 
con = openeo.connect("openeo.dataspace.copernicus.eu")
con.authenticate_oidc()

# Now that we are connected, we can initialize our datacube object with the area of interest 
# and the time range of interest using Sentinel 1 data.
datacube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent={"west": 5.14, "south": 51.17, "east": 5.17, "north": 51.19},
    temporal_extent = ["2021-02-01", "2021-04-30"],
    bands=["B02", "B04", "B08"],
    max_cloud_cover=85,
)

# By filtering as early as possible (directly in load_collection() in this case), 
# we make sure the back-end only loads the data we are interested in and avoid incurring unneeded costs.


#From this data cube, we can now select the individual bands with the DataCube.band() method and rescale the digital number values to physical reflectances:
blue = sentinel2_cube.band("B02") * 0.0001
red = sentinel2_cube.band("B04") * 0.0001
nir = sentinel2_cube.band("B08") * 0.0001


# We now want to compute the enhanced vegetation index and can do that directly with these band variables:
evi_cube = 2.5 * (nir - red) / (nir + 6.0 * red - 7.5 * blue + 1.0)

# Now we can use the compact “band math” feature again to build a binary mask with a simple comparison operation:
# Select the "SCL" band from the data cube
scl_band = s2_scl.band("SCL")
# Build mask to mask out everything but class 4 (vegetation)
mask = (scl_band != 4)

# Before we can apply this mask to the EVI cube we have to resample it, as the “SCL” layer has a “ground sample distance” of 20 meter, while it is 10 meter for the “B02”, “B04” and “B08” bands. We can easily do the resampling by referring directly to the EVI cube.
mask_resampled = mask.resample_cube_spatial(evi_cube)

# Apply the mask to the `evi_cube`
evi_cube_masked = evi_cube.mask(mask_resampled)

# Because GeoTIFF does not support a temporal dimension, we first eliminate it by taking the temporal maximum value for each pixel:
evi_composite = evi_cube.max_time()

# Now we can download this to a local file:
evi_composite.download("evi-composite.tiff")

In [ ]:
import openeo

connection = openeo.connect("openeo.dataspace.copernicus.eu")

In [ ]:
# List collections available on the openEO back-end
connection.list_collection_ids()

# Get detailed metadata of a certain collection
connection.describe_collection("SENTINEL2_L2A")

In [ ]:
connection.authenticate_oidc()

In [ ]:
datacube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent={"west": 5.14, "south": 51.17, "east": 5.17, "north": 51.19},
    temporal_extent = ["2021-02-01", "2021-04-30"],
    bands=["B02", "B04", "B08"],
    max_cloud_cover=85,
)
cube = datacube.apply(lambda x: 0.004 * x - 0.08).max_time()

cube.download("ndvi-max.tiff")

In [ ]:
import openeo

# Connect to openEO back-end.
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

# Load data cube from TERRASCOPE_S2_NDVI_V2 collection.
cube = connection.load_collection(
    "TERRASCOPE_S2_NDVI_V2",
    spatial_extent={"west": 5.05, "south": 51.21, "east": 5.1, "north": 51.23},
    temporal_extent=["2022-05-01", "2022-05-30"],
    bands=["NDVI_10M"],
)
# Rescale digital number to physical values and take temporal maximum.
cube = cube.apply(lambda x: 0.004 * x - 0.08).max_time()

cube.download("ndvi-max.tiff")